In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

print(os.listdir("./input"))

['BBC News Test.csv', 'BBC News Train.csv', 'fig_data_topicmodel.csv']


In [2]:
# Data Extraction and exploration

fig_data = pd.read_csv("./input/fig_data_topicmodel.csv")

In [3]:
fig_data.head()

,url_cleaned
0,droit-finances.commentcamarche.com/faq/15494-s...
1,droit-finances.commentcamarche.com/forum/affic...
2,droit-finances.commentcamarche.com/faq/4843-aa...
3,droit-finances.commentcamarche.com/faq/7527-al...
4,premium.lefigaro.fr/musique/2018/08/01/03006-2...


In [4]:
data_text = fig_data[['url_cleaned']]
data_text['index'] = data_text.index
documents = data_text

In [5]:
print(len(documents))
print(documents[:5])

70329
                                         url_cleaned  index
0  droit-finances.commentcamarche.com/faq/15494-s...      0
1  droit-finances.commentcamarche.com/forum/affic...      1
2  droit-finances.commentcamarche.com/faq/4843-aa...      2
3  droit-finances.commentcamarche.com/faq/7527-al...      3
4  premium.lefigaro.fr/musique/2018/08/01/03006-2...      4


In [6]:
!pip install gensim
!pip install spacy

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')



import spacy

!python -m spacy download fr
#nlp = spacy.load("fr")
#spacy.download('fr_core_web_sm')
#pacy.download('fren_core_web_sm')

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/Users/deepashree/anaconda3/lib/python3.6/site-packages/fr_core_news_sm -->
/Users/deepashree/anaconda3/lib/python3.6/site-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [7]:
#tokenise words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(documents['url_cleaned']))

print(data_words[:1])

[['droit', 'finances', 'commentcamarche', 'com', 'faq', 'salaire', 'une', 'assistante', 'maternelle', 'ce', 'qu', 'il', 'faut', 'savoir']]


In [8]:
#create stop words list

stop_words = set(stopwords.words('french'))

#add words that aren't in the NLTK stopwords list
new_stopwords = ['cette', 'les', 'cet']
new_stopwords_list = stop_words.union(new_stopwords)

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['droit', 'finances', 'commentcamarche', 'com', 'faq', 'salaire', 'une', 'assistante', 'maternelle', 'ce', 'qu', 'il_faut_savoir']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words and len(word) > 3] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('fr', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

[['droit', 'finances', 'commentcamarche', 'salaire', 'assistante', 'maternelle', 'faut_savoir']]


In [12]:
#key is the word and value is the number of times that word occurs in the entire corpus
dictionary = gensim.corpora.Dictionary(data_lemmatized)
print(len(dictionary))

30662


In [13]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 3 times
- words appearing in more than 20% of all documents
'''
dictionary.filter_extremes(no_below=3, no_above=0.20, keep_n= 100000)
print(len(dictionary))

13122


In [14]:
# Create Dictionary
#id2word = corpora.Dictionary(dictionary)

# Create Corpus
#bag of words
corpus = [dictionary.doc2bow(doc) for doc in data_lemmatized]

# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.145*"flash" + 0.144*"filwww" + 0.130*"info" + 0.055*"programme" + 0.024*"article" + 0.021*"jeunes_femme" + 0.021*"video" + 0.015*"homme" + 0.015*"montant" + 0.011*"premier"


Topic: 1 
Words: 0.080*"sant" + 0.058*"medecin" + 0.057*"Monsieur" + 0.052*"photo" + 0.031*"omar" + 0.030*"celebrite" + 0.020*"plus" + 0.017*"bain" + 0.016*"maillot" + 0.015*"traitement"


Topic: 2 
Words: 0.106*"people" + 0.098*"magazin" + 0.070*"star" + 0.065*"television" + 0.032*"week" + 0.029*"caun" + 0.029*"devaste_emmer" + 0.029*"papa_belmondo" + 0.025*"miss" + 0.023*"femme"


Topic: 3 
Words: 0.103*"cuisin" + 0.096*"recette" + 0.060*"gastronomie" + 0.037*"finance" + 0.035*"deco" + 0.032*"droit" + 0.017*"commentcamarche" + 0.017*"commentcamarch" + 0.016*"salon" + 0.011*"mort"


Topic: 4 
Words: 0.137*"actualité" + 0.107*"sport" + 0.053*"direct" + 0.027*"economie" + 0.018*"football" + 0.017*"prix" + 0.016*"tirage" + 0.012*"tendance" + 0.012*"tennis" + 0.011*"choc"


Topic: 5 
Words: 0.140*"

In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.43523387234754

Coherence Score:  0.4386791885652738
